# Tobig's 15기 2주차 Optimization 과제

# Gradient Descent 구현하기

### 1)"..."표시되어 있는 빈 칸을 채워주세요
### 2)강의내용과 코드에 대해 공부한 내용을 마크마운 또는 주석으로 설명해주세요

## 데이터

In [1]:
import pandas as pd
import numpy as np
import random

In [2]:
data = pd.read_csv('assignment_2.csv')
data.head()

,Label,bias,experience,salary
0,1,1,0.7,48000
1,0,1,1.9,48000
2,1,1,2.5,60000
3,0,1,4.2,63000
4,0,1,6.0,76000


## Train Test 데이터 나누기

In [3]:
from sklearn.model_selection import train_test_split

In [4]:
X_train, X_test, y_train, y_test = train_test_split(data.iloc[:, 1:], data.iloc[:, 0], test_size = 0.25, random_state = 0)

In [5]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((150, 3), (50, 3), (150,), (50,))

## Scaling

experience와 salary의 단위, 평균, 분산이 크게 차이나므로 scaler를 사용해 단위를 맞춰줍니다. 

In [6]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
bias_train = X_train["bias"]
bias_train = bias_train.reset_index()["bias"]
X_train = pd.DataFrame(scaler.fit_transform(X_train), columns = X_train.columns)
X_train["bias"] = bias_train
X_train.head()

,bias,experience,salary
0,1,0.187893,-1.143335
1,1,1.185555,0.043974
2,1,-0.310938,-0.351795
3,1,-1.629277,-1.341220
4,1,-1.308600,0.043974


이때 scaler는 X_train에 fit 해주시고, fit한 scaler를 X_test에 적용시켜줍니다.  
똑같이 X_test에다 fit하면 안돼요!

In [7]:
bias_test = X_test["bias"]
bias_test = bias_test.reset_index()["bias"]
X_test = pd.DataFrame(scaler.transform(X_test), columns = X_test.columns) # 그냥 transForm도 가능하다.
X_test["bias"] = bias_test
X_test.head()

,bias,experience,salary
0,1,-1.344231,-0.615642
1,1,0.508570,0.307821
2,1,-0.310938,0.571667
3,1,1.363709,1.956862
4,1,-0.987923,-0.747565


In [8]:
# parameter 개수
N = len(X_train.loc[0])

In [9]:
# 초기 parameter들을 임의로 설정해줍니다.
parameters = np.array([random.random() for i in range(N)])
random_parameters = parameters.copy()
parameters

array([0.13391604, 0.30289478, 0.00871304])

### * LaTeX   

Jupyter Notebook은 LaTeX 문법으로 수식 입력을 지원하고 있습니다.  
LaTeX문법으로 아래의 수식을 완성해주세요  
http://triki.net/apps/3466  
https://jjycjnmath.tistory.com/117

## Dot product
## $z = X_i \theta$

In [10]:
def dot_product(X, parameters):
    z = 0
    for i in range(len(parameters)):
        z +=  X[i] * parameters[i]
    return z

## Logistic Function

## $p =  \frac{1}{1 + e^{-X_i\theta }}$

- sigmoid 함수라고도 부른다.
- 분류 문제를 해결하기 위해 주로 사용한다(0~1의 범위를 갖기 때문)

In [11]:
def logistic(X, parameters):
    z = dot_product(X, parameters)
    p = 1 / ( 1 + np.exp(-z))    # 로지스틱 함수
    return p

In [12]:
logistic(X_train.iloc[1], parameters)

0.6209065113854655

## Object function

Object Function : 목적함수는 Gradient Descent를 통해 최적화 하고자 하는 함수입니다.  
<br>
선형 회귀의 목적함수 == `MSE` 라 한다.
## $l(\theta) = \frac{1}{2}\Sigma(y_i - \theta^{T}X_i)^2$  
참고) $\hat{y_i} = \theta^{T}X_i$
  
로지스틱 회귀의 목적함수를 작성해주세요  
(선형 회귀의 목적함수처럼 강의에 나온대로 작성해주세요. 평균을 고려하는 것은 뒤에 코드에서 수행합니다)
## $l(p) = -\Sigma\left \{y_iX_i\theta - log( 1 + e^{X_i\theta})\right \} $

아래 내용에선 $ -\Sigma\left \{y_i log( \frac {p_i}{1 -p_i}) + log( 1 - p_i)\right \}$ 로 작성하겠습니다.

In [13]:
def minus_log_cross_entropy_i(X, y, parameters):
    p = logistic(X, parameters)
    loss = (y * np.log((p) / (1-p))) + np.log(1-p) # 오차값에 로그를 씌운 값입니다.
    return -np.sum(loss)

In [14]:
def mse_i(X, y, parameters):
    y_hat = dot_product(X, parameters)
    loss = (y - y_hat) ** 2 # Squared error
    return 0.5 * np.sum(loss)

In [15]:
def batch_loss(X_set, y_set, parameters, loss_function, n): #n:현재 배치의 데이터 수
    loss = 0
    for i in range(X_set.shape[0]):
        X = X_set.iloc[i,:]
        y = y_set.iloc[i]
        loss += loss_function(X, y, parameters)
    loss = (loss / n) #loss 평균값으로 계산
    return loss

In [16]:
batch_loss(X_test, y_test, parameters, minus_log_cross_entropy_i, len(X_test))

0.756138453160803

## Gradient
위의 선형회귀의 목적함수 $l(\theta)$와 로지스틱회귀의 목적함수 $l(p)$의 gradient를 작성해주세요  
(위의 목적함수를 참고해서 작성해주세요 = 평균을 고려하는 것은 뒤에 코드에서 수행합니다)

## ${\partial\over{\partial \theta_j}}l(\theta)= -\Sigma(y_i - \theta^{T}X_i)X_{ij} = \Sigma(\theta^{T}X_i - y_i )X_{ij}$
## ${\partial\over{\partial \theta_j}}l(p)= -\Sigma(y_i - p_i)x_{ij}$

In [17]:
def get_gradient_ij(X, y, parameters, j, model):
    if model == 'linear':
        y_hat = dot_product(X, parameters)
        gradient = np.sum(y_hat - y) * X[j]
    else:
        p = logistic(X, parameters)
        gradient = np.sum(p-y) * X[j]
    return gradient

In [18]:
get_gradient_ij(X_train.iloc[0,:], y_train.iloc[0], parameters, 2, 'logistic')

0.5201103577954568

## Batch Gradient
하나의 배치 (X_set, y_set)에 대해 기울기를 구하는 코드를 작성해주세요

In [19]:
def batch_gradient(X_set, y_set, parameters, model):
    gradients = [0 for _ in range(len(parameters))] # 0으로 초기화했다.
    
    for i in range(len(X_set)): # == X_set.shape[0] 과 같다.
        X = X_set.iloc[i,:]
        y = y_set.iloc[i]
        for j in range(len(parameters)):
            gradients[j] += get_gradient_ij(X, y, parameters, j, model)
    
    return gradients

In [20]:
gradients1 = batch_gradient(X_train, y_train, parameters, 'logistic')
gradients1

[37.88452662475613, -6.718267409663314, 22.62416666626021]

## mini-batch
인덱스로 미니 배치 나누기

In [21]:
def batch_idx(X_train, batch_size):
    N = len(X_train)
    nb = (N // batch_size)+1 #number of batch => range 문을 위해 +1을 해줌.
    idx = np.array([i for i in range(N)])
    idx_list = [idx[i*batch_size:(i+1)*batch_size] for i in range(nb) if len(idx[i*batch_size:(i+1)*batch_size]) != 0]
    return idx_list

batch_idx 함수에 대한 설명을 batch_size와 함께 간략하게 작성해주세요  
### 설명
`batch_idx` 는 `X_train` 과 `batch_size` 를 받는다.

`X_train` 은 학습에 사용되는 데이터 행렬이며, `batch_size` 는 미니배치의 크기이다.

함수 내부에서 `N`, `nb` 두개의 변수를 만드는데, 각각 데이터의 개수와, 배치로 나눴을 때 배치 덩어리의 개수이다.

총 데이터의 인덱스 (`N`의 길이)를 `idx` 배열에 저장한다.

그 후에 그것을 `batch_size` 단위로 잘라서 `idx_list` 에 저장한다.

즉 위의 함수는, 전체 데이터로 미니배치 데이터를 만들고, 배치 안에 들어있는 데이터를 인덱스로 표시하여 반환해주는 함수이다.

## Update Parameters
기울기를 갱신하는 코드를 작성해주세요  
(loss와 마찬가지로 기울기를 갱신할 때 배치 사이즈를 고려해 평균으로 갱신해주세요)

In [22]:
def step(parameters, gradients, learning_rate, n): #n:현재 배치의 데이터 수
    for i in range(len(parameters)):
        gradients[i] *= (learning_rate / n) # 평균을 구하고 학습률을 곱함
    
    parameters -= gradients # 전체 그래디언트 업데이트
    return parameters

In [23]:
step(parameters, gradients1, 0.01, len(X_train)) # 갱신된 파라미터 반환함

array([0.13139041, 0.30334266, 0.00720476])

## Gradient Descent
위에서 작성한 함수들을 조합해서 경사하강법 함수를 완성해주세요

- learning_rate: 학습률  
- tolerance: Step이 너무 작아서 더 이상의 학습이 무의미할 때 학습을 멈추는 조건  
- batch: 기울기를 1번 갱신할 때 사용하는 데이터셋  
- epoch: 전체 데이터를 한번 돌 때 1에포크 증가
- num_epoch: 전체 에포크를 몇 번 반복할 것인지
<br>

BGD: "Batch Gradient Descent" : 학습 한 번에 모든 데이터셋 전체를 순회하며 기울기를 구한다.  
SGD: "Stochasitc Gradient Descent" : 랜덤하게 데이터 몇 개(또는 한 개)를 뽑아서 기울기를 구한다.  
MGD: "Mini batch Gradient Descent" : 미니배치를 사용해(데이터의 일부를) 기울기를 구한다. 미니배치 전체를 돌면 학습 한 번에 모든 데이터셋을 돌게 된다. BGD와 SGD의 장점을 합친 방법으로 볼 수 있다.  
<br>
batch_size에 따른 경사하강법의 종류를 적어주세요  
batch_size=1 -> SGD  
batch_size=k -> MGD  
batch_size=whole -> BGD  

In [24]:
def gradient_descent(X_train, y_train, learning_rate = 0.1, num_epoch = 1000, tolerance = 0.00001, model = 'logistic', batch_size = 16):
    stopper = False
    
    N = len(X_train.iloc[0]) # 데이터의 feature 개수이다.
    # 성능의 공정한 평가를 위해서 시드 고정
    np.random.seed(0)
    parameters = np.random.rand(N) # 랜덤한 계수로 초기화했다.
    loss_function = minus_log_cross_entropy_i if model == 'logistic' else mse_i # 모델에 따라 손실 함수가 달라져야 한다.
    loss = 999 # 임의의 loss(엄청 큰 값)
    batch_idx_list = batch_idx(X_train, batch_size) # 모델을 1배치에 해당하는 X행렬 원소의 인덱스를 넣어 놓은 행렬 생성
    
    for epoch in range(num_epoch):
        if stopper:
            break
        for idx in batch_idx_list:
            X_batch = X_train.iloc[idx,]
            y_batch = y_train.iloc[idx]
            gradients = batch_gradient(X_batch, y_batch, parameters, loss_function) # 배치의 그래디언트를 구한다.
            parameters = step(parameters, gradients, learning_rate, batch_size) # 구한 그래디언트를 미분하여 행렬을 갱신한다.
            new_loss = batch_loss(X_batch, y_batch, parameters, loss_function, batch_size) # 손실 함수를 이용하여 손실값을 구한다.
            
            #중단 조건
            if abs(new_loss - loss) < tolerance:
                stopper = True
                print(f"Early Stopped at loss: {new_loss}") # early stop 표기
                break
            loss = new_loss
            
        
        #100epoch마다 학습 상태 출력
        if epoch%100 == 0: #출력이 길게 나오면 check point를 수정해도 됩니다.
            print(f"epoch: {epoch}  loss: {new_loss}  params: {parameters}  gradients: {gradients}")
    
    return parameters

## Implement
경사하강법 함수를 이용해 최적의 모수 찾아보세요. 학습을 진행할 때, Hyper Parameter를 바꿔가면서 학습시켜보세요.

## Logistic Regression

같은 하이퍼파라미터의 모델 간 성능 차이

In [25]:
new_param_bgd = gradient_descent(X_train, y_train, learning_rate = 0.05, batch_size=len(X_train), tolerance = 0.00001, model="logistic")
new_param_bgd

epoch: 0  loss: 0.9685644360044335  params: [0.53290843 0.71000918 0.58745526]  gradients: [0.015905073327806236, 0.005180187730894569, 0.015308116334578011]
epoch: 100  loss: 0.5256474897515783  params: [-0.47666549  0.6217811  -0.39145511]  gradients: [0.005425983334753183, -0.0021837480381650797, 0.005549362855984847]
epoch: 200  loss: 0.4511977556397476  params: [-0.8114574   0.90018792 -0.80401959]  gradients: [0.0020268837935590654, -0.0029220251697490342, 0.0032960135079598634]
epoch: 300  loss: 0.41483251242497177  params: [-0.95435314  1.17642119 -1.09386919]  gradients: [0.001023671830597672, -0.0025766021771631118, 0.0025900367127658754]
epoch: 400  loss: 0.3908947331324743  params: [-1.03607738  1.4149229  -1.33024423]  gradients: [0.0006696510861670546, -0.002207117844355731, 0.0021672074067224985]
epoch: 500  loss: 0.373758731823303  params: [-1.09459009  1.61985188 -1.53062856]  gradients: [0.0005207561756130711, -0.001905971373589264, 0.0018575913364437633]
epoch: 600  

array([-1.2870103 ,  2.33078509, -2.21789111])

In [26]:
new_param_sgd = gradient_descent(X_train, y_train, learning_rate = 0.05, batch_size=len(X_train), tolerance = 0.00001, model="logistic")
new_param_sgd

epoch: 0  loss: 0.9685644360044335  params: [0.53290843 0.71000918 0.58745526]  gradients: [0.015905073327806236, 0.005180187730894569, 0.015308116334578011]
epoch: 100  loss: 0.5256474897515783  params: [-0.47666549  0.6217811  -0.39145511]  gradients: [0.005425983334753183, -0.0021837480381650797, 0.005549362855984847]
epoch: 200  loss: 0.4511977556397476  params: [-0.8114574   0.90018792 -0.80401959]  gradients: [0.0020268837935590654, -0.0029220251697490342, 0.0032960135079598634]
epoch: 300  loss: 0.41483251242497177  params: [-0.95435314  1.17642119 -1.09386919]  gradients: [0.001023671830597672, -0.0025766021771631118, 0.0025900367127658754]
epoch: 400  loss: 0.3908947331324743  params: [-1.03607738  1.4149229  -1.33024423]  gradients: [0.0006696510861670546, -0.002207117844355731, 0.0021672074067224985]
epoch: 500  loss: 0.373758731823303  params: [-1.09459009  1.61985188 -1.53062856]  gradients: [0.0005207561756130711, -0.001905971373589264, 0.0018575913364437633]
epoch: 600  

array([-1.2870103 ,  2.33078509, -2.21789111])

In [27]:
new_param_mgd = gradient_descent(X_train, y_train, learning_rate = 0.05, batch_size=16, num_epoch = 1000, tolerance = 0.00001, model="logistic")
new_param_mgd

epoch: 0  loss: 0.5462287389452337  params: [0.40423489 0.66940135 0.46288441]  gradients: [0.011513694380674963, 0.01055556584070781, 0.01334110015487331]
epoch: 100  loss: 0.09990426559964741  params: [-1.27160695  2.27052645 -2.16434608]  gradients: [0.0018475445848693322, 0.0024697939142438556, 0.003463942367615205]
Early Stopped at loss: 0.3588002232980838


array([-1.4147301 ,  2.77741606, -2.64260998])

같은 하이퍼파라미터로 비교했을 때 MGD가 가장 적은 loss 를 보였다.

이후 MGD의 하이퍼파라미터를 조정해가며 학습을 비교해 보겠다.

In [28]:
# 배치 사이즈 2배
new_param_mgd = gradient_descent(X_train, y_train, learning_rate = 0.05, batch_size=32, num_epoch = 1000, tolerance = 0.00001, model="logistic")

epoch: 0  loss: 0.7801054684429901  params: [0.47816848 0.64454434 0.53211835]  gradients: [0.011799936842640017, 0.008623766736598808, 0.01765607925473053]
epoch: 100  loss: 0.4126957459228419  params: [-0.33559063 -0.16921476 -0.28164075]  gradients: [0.0026848342358628714, -0.00218887136654188, 0.004219010159612238]
epoch: 200  loss: 0.4126850786311187  params: [-0.33570808 -0.16933221 -0.2817582 ]  gradients: [0.0026835397229278165, -0.0021904714670937936, 0.004217039337701465]
epoch: 300  loss: 0.4126850766367315  params: [-0.3357081  -0.16933224 -0.28175823]  gradients: [0.002683539480792918, -0.0021904717663850824, 0.00421703896906922]
epoch: 400  loss: 0.41268507663635834  params: [-0.3357081  -0.16933224 -0.28175823]  gradients: [0.002683539480747618, -0.0021904717664410745, 0.0042170389690002535]
epoch: 500  loss: 0.41268507663635834  params: [-0.3357081  -0.16933224 -0.28175823]  gradients: [0.0026835394807476105, -0.0021904717664410823, 0.004217038969000245]
epoch: 600  los

배치 사이즈가 증가하니까, 오차가 증가하는 것으로 보였다.

한 배치에 처리해야할 양의 데이터가 많아서 오차가 증가하는 것으로 직관적으로 이해할 수 있을 듯 하다.

In [28]:
# 학습률 0.01 에폭 수 증가
new_param_mgd = gradient_descent(X_train, y_train, learning_rate = 0.01, batch_size=16, num_epoch = 2000, tolerance = 0.00001, model="logistic")

epoch: 0  loss: 0.6111239638187248  params: [0.51916809 0.70558299 0.57420036]  gradients: [0.0024131831863511877, 0.0021924542416963916, 0.00275905382281014]
Early Stopped at loss: 0.5068818576097247


In [29]:
# 학습률 0.005 # 에폭 수 증가 3000
new_param_mgd = gradient_descent(X_train, y_train, learning_rate = 0.005, batch_size=16, num_epoch = 3000, tolerance = 0.00001, model="logistic")

epoch: 0  loss: 0.6196808273885228  params: [0.5339465  0.71035941 0.58844676]  gradients: [0.001213173594239148, 0.0011009561104164496, 0.0013848003370405557]
epoch: 100  loss: 0.2197450345412278  params: [-0.43871472  0.60922392 -0.35396112]  gradients: [0.0005154215239430915, 0.000506471626629391, 0.0007247899434964056]
epoch: 200  loss: 0.1622597912104849  params: [-0.78286222  0.86283891 -0.76097751]  gradients: [0.0003115285599354422, 0.000329968400980178, 0.00051818923327476]
epoch: 300  loss: 0.14260389966062684  params: [-0.93301302  1.12630304 -1.04360308]  gradients: [0.00025121502184278924, 0.000283969879598398, 0.0004523278545567537]
epoch: 400  loss: 0.1311948695206934  params: [-1.01794449  1.35745235 -1.27418874]  gradients: [0.0002260340099148344, 0.00026750906417607025, 0.00042013385838201146]
epoch: 500  loss: 0.12306665341035632  params: [-1.07734973  1.55772055 -1.47050078]  gradients: [0.0002124552536762419, 0.0002599031154811892, 0.0003996226379199426]
epoch: 600

In [30]:
# 학습률 0.0001 # 에폭 수 증가 3000
new_param_mgd = gradient_descent(X_train, y_train, learning_rate = 0.0001, batch_size=16, num_epoch = 3000, tolerance = 0.00001, model="logistic")

epoch: 0  loss: 0.6281598628995398  params: [0.5485153  0.71509225 0.60247636]  gradients: [2.4391012815701843e-05, 2.2110256134171673e-05, 2.7797589124863234e-05]
epoch: 100  loss: 0.6109706517013486  params: [0.51888534 0.70551661 0.57394411]  gradients: [2.4098806306849923e-05, 2.1899339927623642e-05, 2.7563453666256634e-05]
epoch: 200  loss: 0.5942031062304017  params: [0.48963756 0.69621638 0.54575383]  gradients: [2.3799509698308466e-05, 2.168062827789173e-05, 2.7320455478470447e-05]
epoch: 300  loss: 0.5778638166515099  params: [0.46077977 0.68720001 0.51791603]  gradients: [2.3493463770867718e-05, 2.1454259871610062e-05, 2.7068777920297084e-05]
epoch: 400  loss: 0.5619585435815306  params: [0.43231945 0.67847568 0.49044096]  gradients: [2.3181055870008754e-05, 2.1220425450616452e-05, 2.680866392169228e-05]
epoch: 500  loss: 0.5464921496014734  params: [0.4042637  0.67005121 0.4633385 ]  gradients: [2.2862718734379334e-05, 2.0979369103906815e-05, 2.654041684700171e-05]
epoch: 60

학습률을 많이 낮추지 않는 이상 드라마틱한 변화는 일어나지 않는 것을 확인할 수 있었다.

학습률이 낮아지면 낮아질수록 loss 가 줄어드는 속도가 변화하는 것을 확인할 수 있었다.

이 경우, 시간에 따라서 학습률을 변화시키는  `RMSProp` 이나 `Adam` 과 같은 옵티마이저가 필요할 것 같다.

### Predict Label

In [31]:
y_predict = []
for i in range(len(y_test)):
    p = logistic(X_test.iloc[i,:], new_param_bgd) # 학습한 bgd를 가지고 예측 결과 산출
    if p> 0.5 :
        y_predict.append(1)
    else :
        y_predict.append(0)
y_predict_random = []
for i in range(len(y_test)):
    p = logistic(X_test.iloc[i,:], random_parameters)
    if p> 0.5 :
        y_predict_random.append(1)
    else :
        y_predict_random.append(0)

### Confusion Matrix

In [32]:
from sklearn.metrics import *

In [33]:
tn, fp, fn, tp = confusion_matrix(y_test, y_predict).ravel()
confusion_matrix(y_test, y_predict)

array([[38,  2],
       [ 4,  6]])

In [34]:
accuracy = (tp+tn) / (tp+fn+fp+tn) # 구한 값으로 비교
print("accuracy:",accuracy)

accuracy: 0.88


## Linear regression
### $y = 0.5 + 2.7x$

### Data

In [35]:
raw_X = np.random.rand(150)
y = 2.7*raw_X + 0.5 + np.random.randn(150)

In [36]:
tmp = np.array([1 for _ in range(150)]) # 편향 1 추가
X = np.vstack((tmp, raw_X)).T
X = pd.DataFrame(X)
y = pd.Series(y)

### Estimation

In [37]:
#정규방정식
theta = np.linalg.inv(np.dot(X.T,X)).dot(X.T).dot(y)
theta

array([0.35767412, 2.75736157])

In [ ]:
#경사하강법
new_param = gradient_descent(X, y, learning_rate = 0.0001, batch_size=16, num_epoch = 1900, tolerance = 0.00001, model="linear" )
# 선형 모델 생성 후 학습
new_param

epoch: 0  loss: 0.9986840948275538  params: [0.5497891  0.71587949]  gradients: [-7.561815494881488e-05, -5.935681988982057e-05]
epoch: 100  loss: 0.9052565600347382  params: [0.6461226  0.78425197]  gradients: [-7.465034046656767e-05, -5.872174005410375e-05]
epoch: 200  loss: 0.8211387891420969  params: [0.74016257 0.85143529]  gradients: [-7.377128836470326e-05, -5.815208962016526e-05]
epoch: 300  loss: 0.7457107531898034  params: [0.83210157 0.91754075]  gradients: [-7.2975121808612e-05, -5.764297702409657e-05]
epoch: 400  loss: 0.6784240107491304  params: [0.92212075 0.98267209]  gradients: [-7.225551075980899e-05, -5.718921990573648e-05]
epoch: 500  loss: 0.6187955057145834  params: [1.01038891 1.04692501]  gradients: [-7.16060244566833e-05, -5.678562327426635e-05]
epoch: 600  loss: 0.5664011394450111  params: [1.09706226 1.11038717]  gradients: [-7.10203725226204e-05, -5.64271657865428e-05]
epoch: 700  loss: 0.520869465471667  params: [1.18228451 1.17313832]  gradients: [-7.04925

In [ ]:
y_hat_NE = theta.dot(X.T) # 정규 방정식의 예측 결과
y_hat_GD = new_param.dot(X.T) # 경사하강법의 예측 결과

### Visualization
시각화를 통해 정규방정식과 경사하강법을 통한 선형회귀를 비교해보세요  
(밑의 코드를 실행만 시키면 됩니다. 추가 코드 x)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(X.iloc[:,1], y, '.k') #산점도
plt.plot(X.iloc[:,1], y_hat_NE, '-b', label = 'NE') #정규방정식
plt.plot(X.iloc[:,1], y_hat_GD, '-r', label = 'GD') #경사하강법
plt.legend()
plt.show()

여러 번 시도해본 결과, 1900에폭에서 최소 오차가 나온 것을 확인할 수 있다.

이 이상 발생하면 과적합 등으로 오차가 다시 커지는 것을 확인할 수 있었다. 이는 학습률에 대한 문제로, 학습률을 수정하면 조금 더 정규방정식에 적합해질 것으로 보인다.

실제 에폭을 달리해서 그래프를 확인해 보았는데, 빨간 선(GD) 가 점점 파란 선(NE)에 가까워지는 것을 확인할 수 있었다.